In [ ]:
import os
import sys
from pathlib import Path
sys.path.insert(1, str(Path(os.getcwd()).parent / "src"))

In [ ]:
import data_loader
from concentr_diffs_pathlength import *
from plotting import *
from mbll_functions import *
from utils import *
from mcs_function import *
from pathlib import Path
import config
import pickle
from scipy.optimize import least_squares
import pmcx
from tqdm.notebook import tqdm
import imageio

import matplotlib.pyplot as plt

In [ ]:
config.gpuid

In [ ]:
config.m_params_path

In the Jacques model, diffuse reflectance is given as $$R_d = \exp(-A\delta \mu_a)$$, where A depends on the ratio of the absorption and scattering $N = \frac{\mu_s'}{\mu_a}$ coefficient, and is modelled as
$$ A(N) = m_1 + m_2\exp(\frac{\ln(N)}{m_3}) $$
and $\delta$ is $$\delta = (3\mu_a(\mu_s' + \mu_a))^{-\frac{1}{2}}$$
A also depends on the refractive index. The $m_i$-parameters in paper and Jacque's website are based on a refractive index of 1.33. For the brain we expect 1.36 (gray matter) to 1.4 (blood vessels). To determine the $m_i$ for these indices, we run Monte Carlo Simulations with reasonable absorption/scattering coefficients and fit the $m_i$ to the obtained values for A.\
For absorption/scattering coefficients we use literature values of gray matter and blood vessel tissue. Additionally, we use some very high values for $N$, as it was done on Jacque's website.

In [ ]:
 # both inclusive
loader = data_loader.DataLoader(None, 450, 999)
# absorption spectra for HbO2, Hbb, oxCCO, redCCO, water and fat
mu_a_matrix = loader.absorption_coefs(
    use_diff_oxycco = False,
    use_water_and_fat = True
)

In [ ]:
sample_wavelengths = np.arange(loader.wavelength_left_cut, loader.wavelength_right_cut, 25)
if sample_wavelengths[-1] != loader.wavelength_right_cut:
    sample_wavelengths = np.append(sample_wavelengths, loader.wavelength_right_cut)[1:]
print(sample_wavelengths)

In [ ]:
g_gm = 0.85
refractive_idx_gm = 1.36
mu_a_gm = data_loader.DataLoader.mu_a_func_gray_matter(sample_wavelengths) / 10
mu_s_gm = data_loader.DataLoader.mu_s_red_func_gray_matter(sample_wavelengths) / 10 / (1 - g_gm)

In [ ]:
g_bv = 0.935
refractive_idx_bv = 1.4
mu_a_bv = data_loader.DataLoader.mu_a_func_blood_vessel(sample_wavelengths) / 10
mu_s_bv = data_loader.DataLoader.mu_s_red_func_blood_vessel(sample_wavelengths) / 10 / (1 - g_bv)

In [ ]:
g_t = data_loader.DataLoader.tissue_parameters["tumor"][2]
refractive_idx_t = data_loader.DataLoader.tissue_parameters["tumor"][3]
mu_a_t = data_loader.DataLoader.mu_a_func_tissue(sample_wavelengths, "tumor") / 10
mu_s_t = data_loader.DataLoader.mu_s_red_func_tissue(sample_wavelengths, "tumor") / 10 / (1-g_t)

In [ ]:
# For refractive index of 1.38 compute the m-parameters based on combined blood and cortex values
g_combined = 0.8675
refractive_idx_combined = 1.38
mu_a_combined = np.concatenate((mu_a_gm, mu_a_bv, mu_a_t))
mu_s_combined = np.concatenate((mu_s_gm * (1-g_gm) / (1-g_combined), mu_s_bv * (1-g_bv) / (1-g_combined), mu_s_t * (1-g_t) / (1-g_combined)))

In [ ]:
# Optionally append very high N values, as it was done on Jacques website
#mu_a_gm = np.append(mu_a_gm, np.repeat((1 - g_gm), 3))
#mu_s_gm = np.append(mu_s_gm, np.array([1e2, 1e3, 5e3]))
#mu_a_bv = np.append(mu_a_bv, np.repeat((1 - g_bv), 3))
#mu_s_bv = np.append(mu_s_bv, np.array([1e2, 1e3, 5e3]))
mu_a_combined = np.append(mu_a_combined, np.repeat((1 - g_combined), 3))
mu_s_combined = np.append(mu_s_combined, np.array([1e2, 1e3, 5e3]))

In [ ]:
mcs_params_total = {
    "gray matter": (mu_a_gm, mu_s_gm, g_gm, refractive_idx_gm),
    "blood vessel": (mu_a_bv, mu_s_bv, g_bv, refractive_idx_bv),
    "tumor": (mu_a_t, mu_s_t, g_t, refractive_idx_t),
    "general": (mu_a_combined, mu_s_combined, g_combined, refractive_idx_combined)
}

In [ ]:
mu_a_combined.shape

In [ ]:
A_vals = dict()
dref_vals = dict()
m_params = dict()
N_vals = dict()

# empty log file
open(config.m_params_path.parent / "log.txt", "w").close()

# make sure to make this a separate array, don't create directly in cfg!
prop = np.array([[0,0,1,1],[0,0,1.2,1.5]])

vol = np.ones((100, 100, 120), dtype="uint8")

cfg = {
    "nphoton": 1e7,
    "maxdetphoton": 1e7,
    "vol": vol,
    "tstart":0,
    "tend":1e-8,
    "tstep":1e-8,
    "autopilot": 1,
    "gpuid": config.gpuid,
    "prop": prop,
    "bc": "ccrcca",#001000",
    "isspecular": 1,
    "srcdir": [0,0,1],
    "srctype": "planar",
    "srcpos": [0, 0, 0],
    "srcparam1": [vol.shape[0], 0, 0, 0], # 3D position of vertex, fourth coordinate is irrelevant
    "srcparam2": [0, vol.shape[1], 0, 0],
    "issrcfrom0": 1,
    #"savedetflag": "dpx",
    "flog": str(config.m_params_path.parent) + "/log.txt",
    "isnormalized": 0,
    "issaveref": 1
}

# necessary to accumulate weights
cfg["vol"][:, :, 0] = 0

def A_residual(m_params, A_vals, N_vals):
    m1, m2, m3 = m_params
    return A_vals - (m1 + m2 * np.exp(np.log(N_vals)/m3))

for tissue, (mu_a_tissue, mu_s_tissue, g_tissue, refractive_idx) in mcs_params_total.items():
    A_vals[tissue] = []
    N_vals[tissue] = []
    dref_vals[tissue] = []
    for mu_a, mu_s in zip(mu_a_tissue, mu_s_tissue):
        cfg["prop"][1, 0] = mu_a
        cfg["prop"][1, 1] = mu_s
        cfg["prop"][1, 2] = g_tissue
        cfg["prop"][1, 3] = refractive_idx
        res = pmcx.mcxlab(cfg)
        #mean_pl = pmcx.utils.meanpath(res["detp"])[0]
        mu_s_red = mu_s * (1 - g_tissue)
        delta = 1 / np.sqrt(3 * mu_a * (mu_a + mu_s_red))
        #A = mean_pl / delta
        dref = np.sum(res["dref"][:, :, 0, :]) / cfg["nphoton"]
        dref_vals[tissue].append(dref)
        A = -np.log(dref) / (delta * mu_a)
        A_vals[tissue].append(A)
        print(f"N: {mu_s_red / mu_a}    A:{A}")
        
    N_vals[tissue] = mu_s_tissue * (1 - g_tissue) / mu_a_tissue
    # initial values by Jacques as initialization
    m = np.array([6.3744, 0.3568, 3.4739])
    ls_res = least_squares(A_residual, m, args=(A_vals[refractive_idx], N_vals[refractive_idx]))
    m_params[tissue] = ls_res.x


In [ ]:
mu_a_gm.shape

Compare diffuse reflectance computed during MCS with dref computed here...

Store the parameters...

In [ ]:
with open(config.m_params_path, "wb") as f:
    pickle.dump((m_params, A_vals, N_vals, dref_vals), f)

Load the parameters...

In [ ]:
with open(config.m_params_path, "rb") as f:
    m_params, A_vals, N_vals, dref_vals = pickle.load(f)

In [ ]:
N_vals

In [ ]:
#refractive_idxs = [1.36, 1.38, 1.4]

fig, axs = plt.subplots(4, 1)

for i, (tissue, (mu_a_tissue, mu_s_tissue, g_tissue, refractive_idx)) in enumerate(mcs_params_total.items()):
    m1, m2, m3 = m_params[tissue]
    #N_vals = mu_s_tissue * (1 - g_tissue) / mu_a_tissue
    N_vals_interp = np.linspace(np.min(N_vals[tissue]), np.max(N_vals[tissue]), 100)
    axs[i].plot(N_vals_interp, m1 + m2*np.exp(np.log(N_vals_interp)/m3))
    axs[i].scatter(N_vals[tissue], A_vals[tissue])
    axs[i].set_xscale("log")
    axs[i].set_title(f"{tissue}")
    axs[i].text(0.01, 0.5, f"m1: {m1}\nm2: {m2}\nm3: {m3}", transform=axs[i].transAxes)

plt.show()